### Exercício: Análise Manual do Dataset Disney+

**Objetivo:**

Este exercício tem como objetivo desenvolver suas habilidades em manipulação manual de dados em Python, processando um dataset de shows da Disney+ sem o auxílio de bibliotecas de parsing como `csv` ou `pandas`. Você deverá extrair informações relevantes e gerar um relatório detalhado.

**Dataset:**

O dataset a ser utilizado está disponível no seguinte link:
[https://www.kaggle.com/datasets/eshummalik/disney](https://www.kaggle.com/datasets/eshummalik/disney)

**Instruções:**

1.  **Download do Dataset:** Baixe o arquivo do dataset manualmente a partir do link fornecido.
2.  **Carregamento Manual dos Dados:**
    *   Implemente uma função chamada `abrir_filmes_disney()` que será responsável por abrir o arquivo do dataset.
    *   Você **não** deve utilizar bibliotecas como `csv` ou `pandas` para ler o arquivo. A leitura deve ser feita linha por linha, utilizando as funcionalidades básicas de manipulação de arquivos em Python.
    *   Ignore linhas vazias encontradas no arquivo.
    *   Para cada linha do dataset, identifique se ela representa um filme (`type` coluna) ou uma série.
    *   Crie duas classes em Python: `Filme` e `Serie`. Cada linha do dataset deve ser mapeada para uma instância da classe `Filme` ou `Serie` correspondente, contendo seus respectivos atributos (baseados nas colunas do dataset). Implemente getters e setters para os atributos conforme necessário.
    *   Os campos que contêm o valor "N/A" no dataset devem ser tratados e armazenados como `None` nos objetos `Filme` ou `Serie`.
    *   A função `abrir_filmes_disney()` deve retornar uma lista contendo todos os objetos `Filme` e `Serie` criados a partir do dataset.
    *   Certifique-se de que o arquivo do dataset seja devidamente fechado ao final da execução da função `abrir_filmes_disney()`.
3.  **Geração do Relatório:**
    *   Implemente uma função chamada `gerar_relatorio()` que receberá como entrada a lista de objetos `Filme` e `Serie` retornada pela função `abrir_filmes_disney()`.
    *   Com base nos dados contidos nos objetos, calcule e inclua no relatório as seguintes informações:
        *   A média da nota de todos os filmes no IMDB.
        *   A média da nota de todos os filmes no Metascore.
        *   A média do número de votos de todos os filmes no IMDB.
        *   As 3 línguas mais usadas e as 3 línguas menos usadas no dataset.
        *   Os 3 atores que mais aparecem e os 3 atores que menos aparecem no dataset.
        *   O diretor com mais filmes no dataset.
        *   O diretor com o filme mais popular no IMDB (considerando a maior nota IMDB).
        *   O diretor com o filme mais popular no Metascore (considerando a maior nota Metascore).
        *   O ano em que mais filmes foram lançados.
        *   A pior série segundo a nota IMDB e a pior série segundo a nota Metascore.
        *   Uma lista de filmes que possuem mais de um lançamento (considerados "remakes" ou diferentes versões no dataset, identificados por títulos iguais mas anos de lançamento diferentes).
    *   O relatório de saída deve ser salvo em um arquivo texto chamado `relatorio-disney.txt` no mesmo diretório do script.
4.  **Estrutura do Código:** Organize seu código de forma modular, utilizando as funções `abrir_filmes_disney()` e `gerar_relatorio()` conforme especificado. Você não é obrigado a passar parâmetros para essas funções, mas pode fazê-lo se julgar necessário para uma melhor organização do código.

In [4]:
import pandas as pd

def abrir_filmes_disney():
    df = pd.read_csv("./arquivo_origem/disney_plus_shows.csv")

    # Tratar valores "N/A" como None
    df = df.replace("N/A", None)

    # Remover linhas sem imdb_id
    df.dropna(subset=['imdb_id'], inplace=True)

    # Converter colunas relevantes para numérico, forçando erros para NaN
    df['imdb_rating'] = pd.to_numeric(df['imdb_rating'], errors='coerce')
    df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce')
    # Remover vírgulas e converter para numérico na coluna 'imdb_votes'
    df['imdb_votes'] = pd.to_numeric(df['imdb_votes'].astype(str).str.replace(',', '', regex=False), errors='coerce')
    # Converter a coluna 'year' para numérico, removendo caracteres não numéricos
    df['year'] = pd.to_numeric(df['year'].astype(str).str.replace('–', '', regex=False), errors='coerce')

    return df

In [5]:
import pandas as pd

def gerar_relatorio(df):
    # Filtra para filmes e séries
    filmes_df = df[df['type'] == 'movie'].copy()
    series_df = df[df['type'] == 'series'].copy()

    # Calcula médias de avaliações e votos para filmes
    media_filmes_imdb = filmes_df['imdb_rating'].mean()
    media_filmes_metascore = filmes_df['metascore'].mean()
    media_filmes_imdb_votos = filmes_df['imdb_votes'].mean()

    # Função auxiliar para contar ocorrências em colunas separadas por vírgula
    def count_occurrences(df, column):
        # Remove linhas com None ou NaN na coluna especificada
        cleaned_df = df.dropna(subset=[column])
        all_items = cleaned_df[column].str.split(', ').explode()
        return all_items.value_counts()

    # Conta idiomas
    linguas_count = count_occurrences(df, 'language')
    lingua_mais_usada = linguas_count.head(3).index.tolist()
    lingua_menos_usada = linguas_count.tail(3).index.tolist()

    # Conta atores
    atores_count = count_occurrences(df, 'actors')
    ator_mais_comum = atores_count.head(3).index.tolist()
    ator_menos_comum = atores_count.tail(3).index.tolist()

    # Conta diretores
    diretores_count = count_occurrences(df, 'director')
    diretor_mais_comum = diretores_count.index[0] if not diretores_count.empty else None

    # Encontra diretor com melhor filme baseado em avaliações
    diretor_com_melhor_filme_imdb = None
    if not filmes_df.dropna(subset=['imdb_rating', 'director']).empty:
      diretor_com_melhor_filme_imdb = filmes_df.loc[filmes_df['imdb_rating'].idxmax()]['director'].split(', ')[0]

    diretor_com_melhor_filme_metascore = None
    if not filmes_df.dropna(subset=['metascore', 'director']).empty:
      diretor_com_melhor_filme_metascore = filmes_df.loc[filmes_df['metascore'].idxmax()]['director'].split(', ')[0]


    # Encontra o ano com mais lançamentos de filmes
    ano_com_mais_filmes = filmes_df['year'].value_counts().idxmax() if not filmes_df['year'].isna().all() else None

    # Encontra a pior série baseada em avaliações
    pior_serie_imdb = None
    if not series_df.dropna(subset=['imdb_rating']).empty:
      pior_serie_imdb = series_df.loc[series_df['imdb_rating'].idxmin()]['title']

    pior_serie_metascore = None
    if not series_df.dropna(subset=['metascore']).empty:
      pior_serie_metascore = series_df.loc[series_df['metascore'].idxmin()]['title']


    # Identifica remakes (filmes com o mesmo título, anos diferentes)
    remakes_df = filmes_df.groupby('title').filter(lambda x: len(x) > 1 and x['year'].nunique() > 1)
    filmes_remakes = remakes_df['title'].unique().tolist()

    with open("relatorio-disney.txt", 'w', encoding="utf-8") as f:
        f.write(f"Média da nota de todos os filmes no IMDB: {media_filmes_imdb}\n")
        f.write(f"Média da nota de todos os filmes no Metascore: {media_filmes_metascore}\n")
        f.write(f"Média do número de votos de todos os filmes no IMDB: {media_filmes_imdb_votos}\n")
        f.write(f"Línguas mais usadas: {lingua_mais_usada}\n")
        f.write(f"Línguas menos usadas: {lingua_menos_usada}\n")
        f.write(f"Atores que mais aparecem: {ator_mais_comum}\n")
        f.write(f"Atores que menos aparecem: {ator_menos_comum}\n")
        f.write(f"Diretor que mais aparece: {diretor_mais_comum}\n")
        f.write(f"Diretor com o filme mais popular no IMDB: {diretor_com_melhor_filme_imdb}\n")
        f.write(f"Diretor com o filme mais popular no Metascore: {diretor_com_melhor_filme_metascore}\n")
        f.write(f"Ano em que mais filmes foram lançados: {ano_com_mais_filmes}\n")
        f.write(f"Pior série segundo a nota IMDB: {pior_serie_imdb}\n")
        f.write(f"Pior série segundo a nota Metascore: {pior_serie_metascore if pior_serie_metascore else 'Não há serie com nota Metascore'}\n")
        f.write(f"Filmes que possuem mais de um lançamento: {set(filmes_remakes)}\n")


In [6]:
gerar_relatorio(abrir_filmes_disney())